<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 3 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [1]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data

--2019-08-19 23:16:28--  https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18171 (18K) [application/x-httpd-php]
Saving to: ‘house-votes-84.data’

house-votes-84.data 100%[===================>]  17.75K  --.-KB/s    in 0.06s   

2019-08-19 23:16:28 (278 KB/s) - ‘house-votes-84.data’ saved [18171/18171]



In [0]:
import pandas as pd
import seaborn as sns
import numpy as np

df = pd.read_csv("house-votes-84.data", names=['party','handicapped-infants','water-project',
                          'budget','physician-fee-freeze', 'el-salvador-aid',
                          'religious-groups','anti-satellite-ban',
                          'aid-to-contras','mx-missile','immigration',
                          'synfuels', 'education', 'right-to-sue','crime','duty-free',
                          'south-africa'])

In [3]:
df.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


In [4]:
#Now, lets turn the table to binary
for vote in df:
  if vote != 'party':
    df[vote] = df[vote].apply(lambda x: 1 if x == 'y' else 0 if x == 'n' else np.NaN)
    
df.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


###2) Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01

In [5]:
df['party'] == 'republican'

0       True
1       True
2      False
3      False
4      False
5      False
6      False
7       True
8       True
9      False
10      True
11      True
12     False
13     False
14      True
15      True
16     False
17     False
18      True
19     False
20     False
21     False
22     False
23     False
24     False
25     False
26     False
27     False
28      True
29     False
       ...  
405     True
406    False
407    False
408    False
409     True
410     True
411    False
412     True
413     True
414    False
415    False
416     True
417    False
418    False
419    False
420     True
421    False
422    False
423    False
424    False
425    False
426    False
427     True
428    False
429    False
430     True
431    False
432     True
433     True
434     True
Name: party, Length: 435, dtype: bool

In [6]:
#get tables for both parties
rep = df[df['party'] == 'republican']
dem = df[df['party'] == 'democrat']
dem = dem.drop('party', axis=1)
rep = rep.drop(['party'], axis=1)
dem.head()
rep.head()

,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
7,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,1.0
8,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
10,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,1.0,0.0,0.0


In [7]:
rep['budget'].head()

0     0.0
1     0.0
7     0.0
8     0.0
10    0.0
Name: budget, dtype: float64

In [0]:
#define function to find if repubs or dems support and issue more
from scipy.stats import ttest_ind, ttest_rel, ttest_1samp
def find_support(column):
  return ttest_ind(rep[column], dem[column], nan_policy='omit')

In [9]:
ttest_ind(rep['budget'], dem['budget'], nan_policy='omit')

Ttest_indResult(statistic=-23.21277691701378, pvalue=2.0703402795404463e-77)

In [10]:
#get results for all the issues
for vote in rep:
  print(vote, find_support(vote))

handicapped-infants Ttest_indResult(statistic=-9.205264294809222, pvalue=1.613440327937243e-18)
water-project Ttest_indResult(statistic=0.08896538137868286, pvalue=0.9291556823993485)
budget Ttest_indResult(statistic=-23.21277691701378, pvalue=2.0703402795404463e-77)
physician-fee-freeze Ttest_indResult(statistic=49.36708157301406, pvalue=1.994262314074344e-177)
el-salvador-aid Ttest_indResult(statistic=21.13669261173219, pvalue=5.600520111729011e-68)
religious-groups Ttest_indResult(statistic=9.737575825219457, pvalue=2.3936722520597287e-20)
anti-satellite-ban Ttest_indResult(statistic=-12.526187929077842, pvalue=8.521033017443867e-31)
aid-to-contras Ttest_indResult(statistic=-18.052093200819733, pvalue=2.82471841372357e-54)
mx-missile Ttest_indResult(statistic=-16.437503268542994, pvalue=5.03079265310811e-47)
immigration Ttest_indResult(statistic=1.7359117329695164, pvalue=0.08330248490425066)
synfuels Ttest_indResult(statistic=-8.293603989407588, pvalue=1.5759322301054064e-15)
educa

In [11]:
print(dem['physician-fee-freeze'].sum(), rep['physician-fee-freeze'].sum())

14.0 163.0


In [12]:
print(dem['anti-satellite-ban'].sum(), rep['anti-satellite-ban'].sum())

200.0 39.0


It looks like most issues were divisive - physician fee freeze being the worst, with Republicans overwhelmingly supporting and Democrats opposing. On the other hand, Democrats supported an anti-satellite ban Republicans opposed. However, the water project was pretty close to bipartisan according to this analysis. (With p = .01)